### 1. 设置proxy

In [1]:
# # 设置代理端口
# import os
# os.environ["http_proxy"] = "http://127.0.0.1:10809"
# os.environ["https_proxy"] = "http://127.0.0.1:10809"

# # 测试下能否访问外网
# import requests
# response = requests.get("http://www.google.com")
# print(response.status_code)

### 2. 尝试获取数据

In [2]:
# !python d:/workspace/qlib/scripts/get_data.py qlib_data --target_dir ~/.qlib/qlib_data/cn_data --region cn

### 尝试使用qrun

In [3]:
# !qrun d:/workspace/qlib/examples/benchmarks/LightGBM/workflow_config_lightgbm_Alpha158.yaml

### 3. 尝试使用qlib.data相关api

In [5]:
import qlib
from qlib.data import D
from qlib.constant import REG_CN

# Initialize Qlib
qlib.init(provider_uri="~/.qlib/qlib_data/cn_data", region=REG_CN)

# 1. Get 2020 trading days
calendar_2020 = D.calendar(start_time="2020-01-01", end_time="2020-12-31")
print("2020 Trading Days (first 5):", calendar_2020[:5])

# 2. Get CSI 300 instruments (filtered by close price > 20)
from qlib.data.filter import ExpressionDFilter
filter_close = ExpressionDFilter(rule_expression="$close>20")  # Filter: close price > 20
csi300_instruments = D.instruments(market="csi300", filter_pipe=[filter_close])
csi300_list = D.list_instruments(instruments=csi300_instruments, as_list=True)
print("CSI 300 Instruments (close > 20, first 5):", csi300_list[:5])

# 3. Get features: close price + 5-day MA for CSI 300 instruments (2020)
features = ["$close", "Mean($close, 5)"]
feature_df = D.features(
    instruments=csi300_instruments,
    fields=features,
    start_time="2020-01-01",
    end_time="2020-01-10",
    freq="day"
)
print("Feature DataFrame (head):")
print(feature_df.head())

[28948:MainThread](2025-09-15 09:58:50,345) INFO - qlib.Initialization - [config.py:452] - default_conf: client.
[28948:MainThread](2025-09-15 09:58:50,348) INFO - qlib.Initialization - [__init__.py:75] - qlib successfully initialized based on client settings.
[28948:MainThread](2025-09-15 09:58:50,349) INFO - qlib.Initialization - [__init__.py:77] - data_path={'__DEFAULT_FREQ': WindowsPath('C:/Users/jay/.qlib/qlib_data/cn_data')}


2020 Trading Days (first 5): [Timestamp('2020-01-02 00:00:00') Timestamp('2020-01-03 00:00:00')
 Timestamp('2020-01-06 00:00:00') Timestamp('2020-01-07 00:00:00')
 Timestamp('2020-01-08 00:00:00')]
CSI 300 Instruments (close > 20, first 5): ['SH600009', 'SH600027', 'SH600030', 'SH600031', 'SH600048']
Feature DataFrame (head):
                          $close  Mean($close, 5)
instrument datetime                              
SH600009   2020-01-02  25.529419        25.679243
           2020-01-03  25.509619        25.643602
           2020-01-06  24.948605        25.534039
           2020-01-07  24.833101        25.361774
           2020-01-08  24.770399        25.118229


### 4. 测试一下expression相关的api demo

In [12]:
import qlib
from qlib.data import D
from qlib.constant import REG_CN
from qlib.data.filter import ExpressionDFilter

# 初始化QLib
qlib.init(provider_uri="~/.qlib/qlib_data/cn_data", region=REG_CN)

# 基础过滤：拆分多条件为多个Filter
filter_close1 = ExpressionDFilter(rule_expression="$close > 10")  # 收盘价>10元
filter_close2 = ExpressionDFilter(rule_expression="$close < 50")  # 收盘价<50元
filter_volume = ExpressionDFilter(rule_expression="$volume > 1000")  # 成交量>1000手

# 获取符合条件的标的
basic_instruments = D.instruments(
    market="csi300",
    filter_pipe=[filter_close1, filter_close2, filter_volume],
    start_time="2024-01-01",
    end_time="2024-05-31"
)

# 输出结果
print(f"【基础过滤】符合条件的标的数量: {len(basic_instruments)}")
print(f"前5个标的: {D.list_instruments(basic_instruments, as_list=True)[:5]}")

[28948:MainThread](2025-09-15 10:17:51,573) INFO - qlib.Initialization - [config.py:452] - default_conf: client.
[28948:MainThread](2025-09-15 10:17:51,576) INFO - qlib.Initialization - [__init__.py:75] - qlib successfully initialized based on client settings.
[28948:MainThread](2025-09-15 10:17:51,578) INFO - qlib.Initialization - [__init__.py:77] - data_path={'__DEFAULT_FREQ': WindowsPath('C:/Users/jay/.qlib/qlib_data/cn_data')}
[28948:MainThread](2025-09-15 10:17:51,579) WARNING - qlib.Provider - [data.py:1153] - The instruments corresponds to a stock pool. Parameters `start_time` and `end_time` does not take effect now.


【基础过滤】符合条件的标的数量: 2
前5个标的: ['SH600009', 'SH600010', 'SH600016', 'SH600030', 'SH600031']


### 5. 使用qlib download 接口 、以及 baostock 等接口去获取近三年的A股的数据。